In [28]:
!pip3 install -q "giskard[llm]" --upgrade


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip3 install -q langchain faiss-cpu pypdf openai tiktoken langchain-openai langchain_chroma

In [3]:
!pip3 install PYPDF2

In [ ]:
!pip3 uninstall giskard
!pip3 install giskard -U

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
from pathlib import Path

import openai
import pandas as pd
from langchain_google_vertexai import VertexAI
from langchain.chains.base import Chain
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA, load_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
import PyPDF2
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from giskard import Dataset, Model, scan, GiskardClient


In [ ]:
llm = VertexAI(model_name="gemini-pro")


In [ ]:
template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

### CONTEXT
{context}

### QUESTION
Question: {question}
"""


In [7]:
def read_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = reader.pages
        text = ''
        for page_num in num_pages:
            text += page_num.extract_text()
    return text

def chunk(document):
  text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n"],
    chunk_size=1024,
    chunk_overlap=200,
    length_function=len,
  )
  splits = text_splitter.create_documents([document])
  return splits

docs = read_pdf("/home/debian/source/repos/BIMPoC/Backend/data/rag_data/BIM.pdf")
splits = chunk(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),persist_directory="./chroma_db")

In [10]:
TEXT_COULUMN_QUERY="query"

In [11]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template)
base_retriever = vectorstore.as_retriever()

In [12]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context")}
)

In [14]:
class RAGModel(Model):
    def model_predict(self, df: pd.DataFrame) -> pd.DataFrame:
        return df[TEXT_COULUMN_QUERY].apply(lambda x: self.model.invoke({"question": x}))

    def save_model(self, path: str):
        out_dest = Path(path)
        # Save the chain object
        self.model.save(out_dest.joinpath("model.json"))

        # Save the FAISS-based retriever
        db = self.model.retriever.vectorstore
        db.save_local(out_dest.joinpath("chroma"))

    @classmethod
    def load_model(cls, path: str) -> Chain:
        src = Path(path)

        db = Chroma(persist_directory=src.joinpath("chroma"), embedding_function=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

        chain = load_chain(src.joinpath("model.json"), retriever=db.as_retriever())
        return chain


In [15]:
giskard_model = RAGModel(
    model=retrieval_augmented_qa_chain,  # A prediction function that encapsulates all the data pre-processing steps and that could be executed with the dataset used by the scan.
    model_type="text_generation",  # Either regression, classification or text_generation.
    name="Question Answering",  # Optional.
    description="This model answers any question about BIM",  # Is used to generate prompts during the scan.
    feature_names=[TEXT_COULUMN_QUERY]  # Default: all columns of your dataset.
)

# Optional: Wrap a dataframe of sample input prompts to validate the model wrapping and to narrow specific tests' queries.
giskard_dataset = Dataset(pd.DataFrame({
    TEXT_COULUMN_QUERY: [
        "What is BIM?",
        "What are different maturity levels of BIM"
    ]
}))


2024-07-12 10:03:55,252 pid:23023 MainThread giskard.datasets.base INFO     Your 'pandas.DataFrame' is successfully wrapped by Giskard's 'Dataset' wrapper class.


/home/debian/source/repos/BIMPoC/pythonenv/lib/python3.10/site-packages/giskard/core/dataset_validation.py:19: UserWarning: You did not provide the optional argument 'target'. 'target' is the column name in df corresponding to the actual target variable (ground truth).
  warning(


2024-07-12 10:19:39,684 pid:23023 Thread-15 (_track) urllib3.connectionpool WARNING  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /track


In [16]:
print(giskard_model.predict(giskard_dataset).prediction)

2024-07-12 10:04:19,133 pid:23023 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'query': 'object'} to {'query': 'object'}
2024-07-12 10:04:28,862 pid:23023 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (2, 1) executed in 0:00:09.738361
[{'response': "BIM is an activity involving a process to combine information and technology, creating a digital representation of a project. It integrates data from multiple sources and evolves alongside the actual project across the entire timeline, encompassing design, construction, and operational use. While BIM is often associated with software applications, it's more accurately defined as a holistic approach to building design, construction, and maintenance. It can be applied to a wide range of construction projects, extending beyond buildings to infrastructure and land surveying.", 'context': [Document(page_content='. SNÆBJÖRNSSON, \nI. KJARTANSDOTTIR, P. NOWAK) \n \nThis manual is about a 

In [19]:
results = scan(giskard_model, giskard_dataset)


🔎 Running scan…
Estimated calls to your model: ~30
Estimated LLM calls for evaluation: 22

2024-07-12 10:06:38,727 pid:23023 MainThread giskard.scanner.logger INFO     Running detectors: ['LLMBasicSycophancyDetector', 'LLMImplausibleOutputDetector']
Running detector LLMBasicSycophancyDetector…
2024-07-12 10:06:39,959 pid:23023 MainThread httpx        INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-07-12 10:06:39,960 pid:23023 MainThread openai._base_client INFO     Retrying request to /chat/completions in 0.774960 seconds
2024-07-12 10:06:40,990 pid:23023 MainThread httpx        INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-07-12 10:06:40,991 pid:23023 MainThread openai._base_client INFO     Retrying request to /chat/completions in 1.867322 seconds
2024-07-12 10:06:43,130 pid:23023 MainThread httpx        INFO     HTTP Request: POST https://api.openai.com/v1/chat/

/home/debian/source/repos/BIMPoC/pythonenv/lib/python3.10/site-packages/giskard/scanner/scanner.py:367: UserWarning: 2 errors were encountered while running detectors. Please check the log to understand what went wrong. You can run the scan again with `raise_exceptions=True` to disable graceful handling.
  warning(


In [20]:
display(results)

<iframe id="scan-130837142496368" srcdoc="<!doctype html>
<html lang="en">

<head>
 <title>Giskard Scan Results</title>
 <base target="_blank">
 <meta charset="utf-8">
 
<style>pre code.hljs{display:block;overflow-x:auto;padding:1em}code.hljs{padding:3px 5px}
/*!
 Theme: GitHub Dark
 Description: Dark theme as seen on github.com
 Author: github.com
 Maintainer: @Hirse
 Updated: 2021-05-15

 Outdated base version: https://github.com/primer/github-syntax-dark
 Current colors taken from GitHub's CSS
*/.hljs{background:#0d1117;color:#c9d1d9}.hljs-doctag,.hljs-keyword,.hljs-meta .hljs-keyword,.hljs-template-tag,.hljs-template-variable,.hljs-type,.hljs-variable.language_{color:#ff7b72}.hljs-title,.hljs-title.class_,.hljs-title.class_.inherited__,.hljs-title.function_{color:#d2a8ff}.hljs-attr,.hljs-attribute,.hljs-literal,.hljs-meta,.hljs-number,.hljs-operator,.hljs-selector-attr,.hljs-selector-class,.hljs-selector-id,.hljs-variable{color:#79c0ff}.hljs-meta .hljs-string,.hljs-regexp,.hljs-string{color:#a5d6ff}.hljs-built_in,.hljs-symbol{color:#ffa657}.hljs-code,.hljs-comment,.hljs-formula{color:#8b949e}.hljs-name,.hljs-quote,.hljs-selector-pseudo,.hljs-selector-tag{color:#7ee787}.hljs-subst{color:#c9d1d9}.hljs-section{color:#1f6feb;font-weight:700}.hljs-bullet{color:#f2cc60}.hljs-emphasis{color:#c9d1d9;font-style:italic}.hljs-strong{color:#c9d1d9;font-weight:700}.hljs-addition{background-color:#033a16;color:#aff5b4}.hljs-deletion{background-color:#67060c;color:#ffdcd7}.hljs-copy-wrapper{overflow:hidden;position:relative}.hljs-copy-button:focus,.hljs-copy-wrapper:hover .hljs-copy-button{transform:translateX(0)}.hljs-copy-button{background-color:#2d2b57;background-color:var(--hljs-theme-background);background-image:url('data:image/svg+xml;charset=utf-8,<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="none" viewBox="0 0 24 24"><path fill="%23fff" fill-rule="evenodd" d="M6 5a1 1 0 0 0-1 1v14a1 1 0 0 0 1 1h12a1 1 0 0 0 1-1V6a1 1 0 0 0-1-1h-2a1 1 0 1 1 0-2h2a3 3 0 0 1 3 3v14a3 3 0 0 1-3 3H6a3 3 0 0 1-3-3V6a3 3 0 0 1 3-3h2a1 1 0 0 1 0 2H6Z" clip-rule="evenodd"/><path fill="%23fff" fill-rule="evenodd" d="M7 3a2 2 0 0 1 2-2h6a2 2 0 0 1 2 2v2a2 2 0 0 1-2 2H9a2 2 0 0 1-2-2V3Zm8 0H9v2h6V3Z" clip-rule="evenodd"/></svg>');background-position:50%;background-repeat:no-repeat;border:1px solid #ffffff22;border-radius:.25rem;color:#fff;height:2rem;position:absolute;right:1em;text-indent:-9999px;top:1em;transition:background-color .2s ease,transform .2s ease-out;width:2rem}.hljs-copy-button:hover{border-color:#ffffff44}.hljs-copy-button:active{border-color:#ffffff66}.hljs-copy-button[data-copied=true]{background-image:none;text-indent:0;width:auto}@media (prefers-reduced-motion){.hljs-copy-button{transition:none}}.hljs-copy-alert{clip:rect(0 0 0 0);-webkit-clip-path:inset(50%);clip-path:inset(50%);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px}
/*! tailwindcss v3.3.2 | MIT License | https://tailwindcss.com*/*,:after,:before{border:0 solid #e5e7eb;box-sizing:border-box}:after,:before{--tw-content:""}html{-webkit-text-size-adjust:100%;font-feature-settings:normal;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-variation-settings:normal;line-height:1.5;-moz-tab-size:4;-o-tab-size:4;tab-size:4}body{line-height:inherit;margin:0}hr{border-top-width:1px;color:inherit;height:0}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,pre,samp{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{border-collaps

In [28]:
test_suite = results.generate_test_suite("Test suite generated by scan")
test_suite.run()

2024-06-19 09:46:18,240 pid:40819 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'query': 'object'} to {'query': 'object'}
2024-06-19 09:46:18,242 pid:40819 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (10, 1) executed in 0:00:00.008270
Executed 'Output plausibility' with arguments {'model': <__main__.RAGModel object at 0x70051df16c80>, 'dataset': <giskard.datasets.base.Dataset object at 0x70051df47eb0>}: 
               Test failed
               Metric: 8
               
               
2024-06-19 09:46:45,835 pid:40819 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'query': 'object'} to {'query': 'object'}
2024-06-19 09:46:45,840 pid:40819 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (10, 1) executed in 0:00:00.021899
2024-06-19 09:46:45,853 pid:40819 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'query': 'object'} to {'query': 'object'}
2

<TestSuiteResult (failed)>

In [29]:
full_results = scan(giskard_model, giskard_dataset)

🔎 Running scan…
Estimated calls to your model: ~365
Estimated LLM calls for evaluation: 148

2024-06-19 09:51:56,197 pid:40819 MainThread giskard.scanner.logger INFO     Running detectors: ['LLMBasicSycophancyDetector', 'LLMCharsInjectionDetector', 'LLMHarmfulContentDetector', 'LLMImplausibleOutputDetector', 'LLMInformationDisclosureDetector', 'LLMOutputFormattingDetector', 'LLMPromptInjectionDetector', 'LLMStereotypesDetector', 'LLMFaithfulnessDetector']
Running detector LLMBasicSycophancyDetector…
2024-06-19 09:52:22,301 pid:40819 MainThread httpx        INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-19 09:52:22,312 pid:40819 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'query': 'object'} to {'query': 'object'}
2024-06-19 09:52:22,711 pid:40819 ThreadPoolExecutor-98_0 httpx        INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-06-19 09:52:24,059 pid:40819 ThreadPool

2024-06-19 09:54:32,202 pid:40819 MainThread matplotlib.font_manager INFO     Failed to extract font properties from /usr/share/fonts/truetype/noto/NotoColorEmoji.ttf: In FT2Font: Can not load face (unknown file format; error code 0x2)
2024-06-19 09:54:32,378 pid:40819 MainThread matplotlib.font_manager INFO     generated new fontManager
2024-06-19 09:55:20,485 pid:40819 MainThread giskard.scanner.logger INFO     LLMCharsInjectionDetector: Tested `query` for special char injection `\r`	Fail rate = 0.000	Vulnerable = False
2024-06-19 09:55:20,510 pid:40819 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'query': 'object'} to {'query': 'object'}
2024-06-19 09:55:21,097 pid:40819 ThreadPoolExecutor-180_0 httpx        INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-06-19 09:55:22,119 pid:40819 ThreadPoolExecutor-182_0 httpx        INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Reques

/home/debian/source/repos/BIMPoC/pythonenv/lib/python3.10/site-packages/giskard/scanner/scanner.py:367: UserWarning: 3 errors were encountered while running detectors. Please check the log to understand what went wrong. You can run the scan again with `raise_exceptions=True` to disable graceful handling.
  warning(


2024-06-19 10:10:11,078 pid:40819 Thread-18 (_track) urllib3.connectionpool WARNING  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /track
2024-06-19 10:11:20,710 pid:40819 Thread-21 (_track) urllib3.connectionpool WARNING  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /track


In [30]:
display(full_results)

<iframe id="scan-123167216748528" srcdoc="<!doctype html>
<html lang="en">

<head>
 <title>Giskard Scan Results</title>
 <base target="_blank">
 <meta charset="utf-8">
 
<style>pre code.hljs{display:block;overflow-x:auto;padding:1em}code.hljs{padding:3px 5px}
/*!
 Theme: GitHub Dark
 Description: Dark theme as seen on github.com
 Author: github.com
 Maintainer: @Hirse
 Updated: 2021-05-15

 Outdated base version: https://github.com/primer/github-syntax-dark
 Current colors taken from GitHub's CSS
*/.hljs{background:#0d1117;color:#c9d1d9}.hljs-doctag,.hljs-keyword,.hljs-meta .hljs-keyword,.hljs-template-tag,.hljs-template-variable,.hljs-type,.hljs-variable.language_{color:#ff7b72}.hljs-title,.hljs-title.class_,.hljs-title.class_.inherited__,.hljs-title.function_{color:#d2a8ff}.hljs-attr,.hljs-attribute,.hljs-literal,.hljs-meta,.hljs-number,.hljs-operator,.hljs-selector-attr,.hljs-selector-class,.hljs-selector-id,.hljs-variable{color:#79c0ff}.hljs-meta .hljs-string,.hljs-regexp,.hljs-string{color:#a5d6ff}.hljs-built_in,.hljs-symbol{color:#ffa657}.hljs-code,.hljs-comment,.hljs-formula{color:#8b949e}.hljs-name,.hljs-quote,.hljs-selector-pseudo,.hljs-selector-tag{color:#7ee787}.hljs-subst{color:#c9d1d9}.hljs-section{color:#1f6feb;font-weight:700}.hljs-bullet{color:#f2cc60}.hljs-emphasis{color:#c9d1d9;font-style:italic}.hljs-strong{color:#c9d1d9;font-weight:700}.hljs-addition{background-color:#033a16;color:#aff5b4}.hljs-deletion{background-color:#67060c;color:#ffdcd7}.hljs-copy-wrapper{overflow:hidden;position:relative}.hljs-copy-button:focus,.hljs-copy-wrapper:hover .hljs-copy-button{transform:translateX(0)}.hljs-copy-button{background-color:#2d2b57;background-color:var(--hljs-theme-background);background-image:url('data:image/svg+xml;charset=utf-8,<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="none" viewBox="0 0 24 24"><path fill="%23fff" fill-rule="evenodd" d="M6 5a1 1 0 0 0-1 1v14a1 1 0 0 0 1 1h12a1 1 0 0 0 1-1V6a1 1 0 0 0-1-1h-2a1 1 0 1 1 0-2h2a3 3 0 0 1 3 3v14a3 3 0 0 1-3 3H6a3 3 0 0 1-3-3V6a3 3 0 0 1 3-3h2a1 1 0 0 1 0 2H6Z" clip-rule="evenodd"/><path fill="%23fff" fill-rule="evenodd" d="M7 3a2 2 0 0 1 2-2h6a2 2 0 0 1 2 2v2a2 2 0 0 1-2 2H9a2 2 0 0 1-2-2V3Zm8 0H9v2h6V3Z" clip-rule="evenodd"/></svg>');background-position:50%;background-repeat:no-repeat;border:1px solid #ffffff22;border-radius:.25rem;color:#fff;height:2rem;position:absolute;right:1em;text-indent:-9999px;top:1em;transition:background-color .2s ease,transform .2s ease-out;width:2rem}.hljs-copy-button:hover{border-color:#ffffff44}.hljs-copy-button:active{border-color:#ffffff66}.hljs-copy-button[data-copied=true]{background-image:none;text-indent:0;width:auto}@media (prefers-reduced-motion){.hljs-copy-button{transition:none}}.hljs-copy-alert{clip:rect(0 0 0 0);-webkit-clip-path:inset(50%);clip-path:inset(50%);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px}
/*! tailwindcss v3.3.2 | MIT License | https://tailwindcss.com*/*,:after,:before{border:0 solid #e5e7eb;box-sizing:border-box}:after,:before{--tw-content:""}html{-webkit-text-size-adjust:100%;font-feature-settings:normal;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-variation-settings:normal;line-height:1.5;-moz-tab-size:4;-o-tab-size:4;tab-size:4}body{line-height:inherit;margin:0}hr{border-top-width:1px;color:inherit;height:0}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,pre,samp{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{border-collaps

In [32]:
## Generation Metrics

## ROGUE evaluates how good a machine is at summarization by considering common words and sequences between the llm and reference summaries

## BLEU evaluates how the generated text matches the reference it mainly considers the precision of word overlap and could score the summary a
## bit lower because it is only considering the word overlap

## METEOR is more comprehensive as it considers more factors like stemming and synonyms and it considers the overall quality of the sentence
## Might give a higher score because its more flexible when considering the synonyms in an example 

In [ ]:
import vertexai
vertexai.init(project="gemini-api-428204", location="us-central1")


In [44]:
from langchain import FAISS, PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Prepare vector store (FAISS) with IPPC report
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, add_start_index=True)
loader = PyPDFLoader("https://www.ipcc.ch/report/ar6/syr/downloads/report/IPCC_AR6_SYR_LongerReport.pdf")
db = FAISS.from_documents(loader.load_and_split(text_splitter), GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

# Prepare QA chain
PROMPT_TEMPLATE = """You are the Climate Assistant, a helpful AI assistant made by Giskard.
Your task is to answer common questions on climate change.
You will be given a question and relevant excerpts from the IPCC Climate Change Synthesis Report (2023).
Please provide short and clear answers based on the provided context. Be polite and helpful.

Context:
{context}

Question:
{question}

Your answer:
"""

prompt = PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["question", "context"])
climate_qa_chain = RetrievalQA.from_llm(llm=llm, retriever=db.as_retriever(), prompt=prompt)

# Test that everything works
climate_qa_chain.invoke({"query": "Is sea level rise avoidable? When will it stop?"})

2024-07-12 10:30:26,793 pid:23023 MainThread pypdf._reader WARNING  Ignoring wrong pointing object 110 0 (offset 0)
2024-07-12 10:30:26,797 pid:23023 MainThread pypdf._reader WARNING  Ignoring wrong pointing object 531 0 (offset 0)
2024-07-12 10:30:26,800 pid:23023 MainThread pypdf._reader WARNING  Ignoring wrong pointing object 968 0 (offset 0)
2024-07-12 10:30:26,801 pid:23023 MainThread pypdf._reader WARNING  Ignoring wrong pointing object 998 0 (offset 0)
2024-07-12 10:30:26,802 pid:23023 MainThread pypdf._reader WARNING  Ignoring wrong pointing object 1034 0 (offset 0)
2024-07-12 10:30:26,803 pid:23023 MainThread pypdf._reader WARNING  Ignoring wrong pointing object 1221 0 (offset 0)
2024-07-12 10:30:26,804 pid:23023 MainThread pypdf._reader WARNING  Ignoring wrong pointing object 1224 0 (offset 0)
2024-07-12 10:30:26,804 pid:23023 MainThread pypdf._reader WARNING  Ignoring wrong pointing object 1226 0 (offset 0)
2024-07-12 10:30:26,805 pid:23023 MainThread pypdf._reader WARNING  

{'query': 'Is sea level rise avoidable? When will it stop?',
 'result': 'Sea level rise due to climate change is unfortunately unavoidable in the coming centuries due to continuous ocean warming and melting ice sheets. The impacts we experience will persist for thousands of years. Though we cannot avoid it entirely, we can work to limit its effects by mitigating climate change and adapting to its unavoidable consequences. \n \n According to the IPCC report, global mean sea level is expected to keep rising throughout this century, causing regional relative sea level rise across large coastal areas. The severity of the sea level rise depends on our emission levels - lower emissions mean slower and less drastic changes.\n \n Even with the most optimistic emission scenarios, the IPCC projects an increase of global mean sea level by anywhere from 0.28 to 0.55 meters by 2100. Under worst-case scenarios, this figure could climb as high as 0.63 to 1.01 meters. \n \n Notably, these are just ave

In [6]:
import google.generativeai as genai

from giskard.llm.client.gemini import GeminiClient

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

giskard.llm.set_default_client(GeminiClient())


NameError: name 'os' is not defined

In [46]:
import giskard
import pandas as pd

def model_predict(df: pd.DataFrame):
    """Wraps the LLM call in a simple Python function.

    The function takes a pandas.DataFrame containing the input variables needed
    by your model, and must return a list of the outputs (one for each row).
    """
    return [climate_qa_chain.invoke({"query": question}) for question in df["question"]]


# Don’t forget to fill the `name` and `description`: they are used by Giskard
# to generate domain-specific tests.
giskard_model = giskard.Model(
    model=model_predict,
    model_type="text_generation",
    name="Climate Change Question Answering",
    description="This model answers any question about climate change based on IPCC reports",
    feature_names=["question"],
)


2024-07-12 10:31:12,952 pid:23023 MainThread giskard.models.automodel INFO     Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


In [47]:
examples = [
    "According to the IPCC report, what are key risks in the Europe?",
    "Is sea level rise avoidable? When will it stop?",
]
giskard_dataset = giskard.Dataset(pd.DataFrame({"question": examples}), target=None)

print(giskard_model.predict(giskard_dataset).prediction)


2024-07-12 10:31:14,218 pid:23023 MainThread giskard.datasets.base INFO     Your 'pandas.DataFrame' is successfully wrapped by Giskard's 'Dataset' wrapper class.
2024-07-12 10:31:14,224 pid:23023 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-07-12 10:31:23,160 pid:23023 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (2, 1) executed in 0:00:08.941191
[{'query': 'According to the IPCC report, what are key risks in the Europe?', 'result': "## Key Risks in Europe due to Climate Change \n\nAccording to the IPCC report's section on Long-term Climate and Development Futures, Europe faces several significant risks due to climate change. These include:\n\n* **Risks to people, economies, and infrastructure**: \n    * Coastal and inland flooding.\n    * Stress and mortality due to increasing temperatures and heat extremes. \n* **Environmental damage**:\n    * Disruption of marine and ter

In [48]:
report = giskard.scan(giskard_model, giskard_dataset, only="hallucination")

🔎 Running scan…
Estimated calls to your model: ~30
Estimated LLM calls for evaluation: 22

2024-07-12 10:31:42,914 pid:23023 MainThread giskard.scanner.logger INFO     Running detectors: ['LLMBasicSycophancyDetector', 'LLMImplausibleOutputDetector']
Running detector LLMBasicSycophancyDetector…
2024-07-12 10:31:42,943 pid:23023 MainThread root         WARNING  Unsupported seed, ignoring.
2024-07-12 10:31:42,944 pid:23023 MainThread root         WARNING  Unsupported format 'json', ignoring.
2024-07-12 10:31:44,697 pid:23023 MainThread giskard.scanner.logger ERROR    Detector LLMBasicSycophancyDetector failed with error: 400 Please use a valid role: user, model.
Traceback (most recent call last):
  File "/home/debian/source/repos/BIMPoC/pythonenv/lib/python3.10/site-packages/giskard/scanner/scanner.py", line 152, in _run_detectors
    try:
  File "/home/debian/source/repos/BIMPoC/pythonenv/lib/python3.10/site-packages/giskard/scanner/llm/llm_basic_sycophancy_detector.py", line 85, in run


/home/debian/source/repos/BIMPoC/pythonenv/lib/python3.10/site-packages/giskard/scanner/scanner.py:367: UserWarning: 2 errors were encountered while running detectors. Please check the log to understand what went wrong. You can run the scan again with `raise_exceptions=True` to disable graceful handling.
  if errors:


2024-07-12 10:47:28,808 pid:23023 Thread-121 (_track) urllib3.connectionpool WARNING  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /track
2024-07-12 10:47:28,813 pid:23023 Thread-124 (_track) urllib3.connectionpool WARNING  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /track


In [1]:
import giskard
from openai import OpenAI
from giskard.llm.client.openai import OpenAIClient

# Setup the Ollama client with API key and base URL
_client = OpenAI(base_url="http://localhost:11434/v1/", api_key="ollama")
oc = OpenAIClient(model="phi3", client=_client)
giskard.llm.set_default_client(oc)


/home/debian/source/repos/BIMPoC/pythonenv/lib/python3.10/site-packages/giskard/core/validation.py:31: RuntimeWarning: fields may not start with an underscore, ignoring "_args"
  validated_func = validate_arguments(func, config={"arbitrary_types_allowed": True})
/home/debian/source/repos/BIMPoC/pythonenv/lib/python3.10/site-packages/giskard/core/validation.py:31: RuntimeWarning: fields may not start with an underscore, ignoring "_kwargs"
  validated_func = validate_arguments(func, config={"arbitrary_types_allowed": True})


In [3]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import pandas as pd

phi_model = ChatOllama(model="phi3")
prompt = ChatPromptTemplate.from_template("Answer the users query {query}")
phi_chain = prompt | phi_model | StrOutputParser()


def model_predict(df: pd.DataFrame):
    """Wraps the LLM call in a simple Python function.

    The function takes a pandas.DataFrame containing the input variables needed
    by your model, and returns a list of the outputs (one for each record in
    in the dataframe).
    """
    return [phi_chain.invoke({"query": question})for question in df["question"].values]


# Create a giskard.Model object. Don’t forget to fill the `name` and `description`
# parameters: they will be used by our scan to generate domain-specific tests.
giskard_model = giskard.Model(
    model=model_predict,  # our model function
    model_type="text_generation",
    name="General Question Answering",
    description="This model answers any question asked by the user",
    feature_names=["question"],  # input variables needed by your model
)

2024-07-12 11:59:00,654 pid:76306 MainThread giskard.models.automodel INFO     Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


In [4]:
examples = [
    "What are large language models?",
    "What is BIM?",
]
giskard_dataset_phi = giskard.Dataset(pd.DataFrame({"question": examples}), target=None)



2024-07-12 11:59:02,554 pid:76306 MainThread giskard.datasets.base INFO     Your 'pandas.DataFrame' is successfully wrapped by Giskard's 'Dataset' wrapper class.


In [5]:
scan_results = giskard.scan(giskard_model,giskard_dataset_phi)
display(scan_results)  # in your notebook


🔎 Running scan…
Estimated calls to your model: ~365
Estimated LLM calls for evaluation: 148

2024-07-12 11:59:05,692 pid:76306 MainThread giskard.scanner.logger INFO     Running detectors: ['LLMBasicSycophancyDetector', 'LLMCharsInjectionDetector', 'LLMHarmfulContentDetector', 'LLMImplausibleOutputDetector', 'LLMInformationDisclosureDetector', 'LLMOutputFormattingDetector', 'LLMPromptInjectionDetector', 'LLMStereotypesDetector', 'LLMFaithfulnessDetector']
Running detector LLMBasicSycophancyDetector…
2024-07-12 12:01:28,275 pid:76306 MainThread httpx        INFO     HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-12 12:01:28,282 pid:76306 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-07-12 12:14:33,085 pid:76306 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (8, 1) executed in 0:13:04.806951
2024-07-12 12:14:33,125 pid:76306 MainThread gi

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

2024-07-12 13:31:00,930 pid:76306 MainThread giskard.scanner.logger INFO     LLMCharsInjectionDetector: Tested `question` for special char injection `\r`	Fail rate = 1.000	Vulnerable = True
2024-07-12 13:31:00,977 pid:76306 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}


2024-07-12 13:35:02,461 pid:76306 Thread-8 (_track) urllib3.connectionpool WARNING  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /track
2024-07-12 13:41:37,058 pid:76306 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (1, 1) executed in 0:10:36.086334
2024-07-12 13:41:37,106 pid:76306 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-07-12 13:50:15,776 pid:76306 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (1, 1) executed in 0:08:38.679091
2024-07-12 13:50:22,959 pid:76306 MainThread giskard.scanner.logger INFO     LLMCharsInjectionDetector: Tested `question` for special char injection `\x08`	Fail rate = 0.000	Vulnerable = False
LLMCharsInjectionDetector: 1 issue detected. (Took 0:31:05.168584)
Running detector LLMHarmfu

/home/debian/source/repos/BIMPoC/pythonenv/lib/python3.10/site-packages/giskard/scanner/scanner.py:368: UserWarning: 3 errors were encountered while running detectors. Please check the log to understand what went wrong. You can run the scan again with `raise_exceptions=True` to disable graceful handling.
  warning(


<iframe id="scan-125497247653872" srcdoc="<!doctype html>
<html lang="en">

<head>
 <title>Giskard Scan Results</title>
 <base target="_blank">
 <meta charset="utf-8">
 
<style>pre code.hljs{display:block;overflow-x:auto;padding:1em}code.hljs{padding:3px 5px}
/*!
 Theme: GitHub Dark
 Description: Dark theme as seen on github.com
 Author: github.com
 Maintainer: @Hirse
 Updated: 2021-05-15

 Outdated base version: https://github.com/primer/github-syntax-dark
 Current colors taken from GitHub's CSS
*/.hljs{background:#0d1117;color:#c9d1d9}.hljs-doctag,.hljs-keyword,.hljs-meta .hljs-keyword,.hljs-template-tag,.hljs-template-variable,.hljs-type,.hljs-variable.language_{color:#ff7b72}.hljs-title,.hljs-title.class_,.hljs-title.class_.inherited__,.hljs-title.function_{color:#d2a8ff}.hljs-attr,.hljs-attribute,.hljs-literal,.hljs-meta,.hljs-number,.hljs-operator,.hljs-selector-attr,.hljs-selector-class,.hljs-selector-id,.hljs-variable{color:#79c0ff}.hljs-meta .hljs-string,.hljs-regexp,.hljs-string{color:#a5d6ff}.hljs-built_in,.hljs-symbol{color:#ffa657}.hljs-code,.hljs-comment,.hljs-formula{color:#8b949e}.hljs-name,.hljs-quote,.hljs-selector-pseudo,.hljs-selector-tag{color:#7ee787}.hljs-subst{color:#c9d1d9}.hljs-section{color:#1f6feb;font-weight:700}.hljs-bullet{color:#f2cc60}.hljs-emphasis{color:#c9d1d9;font-style:italic}.hljs-strong{color:#c9d1d9;font-weight:700}.hljs-addition{background-color:#033a16;color:#aff5b4}.hljs-deletion{background-color:#67060c;color:#ffdcd7}.hljs-copy-wrapper{overflow:hidden;position:relative}.hljs-copy-button:focus,.hljs-copy-wrapper:hover .hljs-copy-button{transform:translateX(0)}.hljs-copy-button{background-color:#2d2b57;background-color:var(--hljs-theme-background);background-image:url('data:image/svg+xml;charset=utf-8,<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="none" viewBox="0 0 24 24"><path fill="%23fff" fill-rule="evenodd" d="M6 5a1 1 0 0 0-1 1v14a1 1 0 0 0 1 1h12a1 1 0 0 0 1-1V6a1 1 0 0 0-1-1h-2a1 1 0 1 1 0-2h2a3 3 0 0 1 3 3v14a3 3 0 0 1-3 3H6a3 3 0 0 1-3-3V6a3 3 0 0 1 3-3h2a1 1 0 0 1 0 2H6Z" clip-rule="evenodd"/><path fill="%23fff" fill-rule="evenodd" d="M7 3a2 2 0 0 1 2-2h6a2 2 0 0 1 2 2v2a2 2 0 0 1-2 2H9a2 2 0 0 1-2-2V3Zm8 0H9v2h6V3Z" clip-rule="evenodd"/></svg>');background-position:50%;background-repeat:no-repeat;border:1px solid #ffffff22;border-radius:.25rem;color:#fff;height:2rem;position:absolute;right:1em;text-indent:-9999px;top:1em;transition:background-color .2s ease,transform .2s ease-out;width:2rem}.hljs-copy-button:hover{border-color:#ffffff44}.hljs-copy-button:active{border-color:#ffffff66}.hljs-copy-button[data-copied=true]{background-image:none;text-indent:0;width:auto}@media (prefers-reduced-motion){.hljs-copy-button{transition:none}}.hljs-copy-alert{clip:rect(0 0 0 0);-webkit-clip-path:inset(50%);clip-path:inset(50%);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px}
/*! tailwindcss v3.3.2 | MIT License | https://tailwindcss.com*/*,:after,:before{border:0 solid #e5e7eb;box-sizing:border-box}:after,:before{--tw-content:""}html{-webkit-text-size-adjust:100%;font-feature-settings:normal;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-variation-settings:normal;line-height:1.5;-moz-tab-size:4;-o-tab-size:4;tab-size:4}body{line-height:inherit;margin:0}hr{border-top-width:1px;color:inherit;height:0}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,pre,samp{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{border-collaps

2024-07-12 18:51:11,016 pid:76306 Thread-16 (_track) urllib3.connectionpool WARNING  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /track
2024-07-12 18:52:43,175 pid:76306 Thread-18 (_track) urllib3.connectionpool WARNING  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /track


In [61]:
!pip3 -q install google-generativeai


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [63]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ['GEMINI_API_KEY'])

model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content('Teach me about how an LLM works')

print(response.text)


## Understanding LLMs: A Simplified Explanation

Imagine a massive, interconnected network of words and their relationships. That's essentially what a Large Language Model (LLM) is. Here's a breakdown:

**1. The Data:** LLMs are trained on vast amounts of text data, like books, articles, code, and websites. This data teaches the model the patterns, structures, and relationships within human language.

**2. The Network:** The model consists of a complex network of artificial neurons, similar to the human brain. These neurons are connected and interact to process information.

**3. Learning the Patterns:** During training, the LLM analyzes the data and learns to predict the next word in a sequence based on the preceding words. This process involves assigning probabilities to different words based on their context.

**4. Generating Text:** When you provide an input prompt, the LLM uses its learned knowledge to generate text that aligns with the prompt's context and style. It does this by 